In [149]:
import requests
import pandas as pd
from datetime import datetime
from tqdm import tqdm
from matplotlib import pyplot as plt
import mysql.connector
from mysql.connector import Error
import pandas as pd
import nltk

Api key:  741a381b7ce1463eb1fb6c41d42c68fd

In [150]:
# a function to extract the sources I'll be analyzing. I'll focus on the english ones
def getSources():
    source_url = 'https://newsapi.org/v1/sources?language=en'
    response = requests.get(source_url).json()
    sources = []
    for source in response['sources']:
        sources.append(source['id'])
    return sources

# Fuentes de noticias

In [151]:
#Obtiene las fuentes de donde se tendrán las noticias
sources = getSources()
print('number of sources :', len(sources))
print('sources :', sources)

('number of sources :', 60)
('sources :', [u'abc-news-au', u'al-jazeera-english', u'ars-technica', u'associated-press', u'bbc-news', u'bbc-sport', u'bloomberg', u'breitbart-news', u'business-insider', u'business-insider-uk', u'buzzfeed', u'cnbc', u'cnn', u'daily-mail', u'engadget', u'entertainment-weekly', u'espn', u'espn-cric-info', u'financial-times', u'football-italia', u'fortune', u'four-four-two', u'fox-sports', u'google-news', u'hacker-news', u'ign', u'independent', u'mashable', u'metro', u'mirror', u'mtv-news', u'mtv-news-uk', u'national-geographic', u'new-scientist', u'newsweek', u'new-york-magazine', u'nfl-news', u'polygon', u'recode', u'reddit-r-all', u'reuters', u'talksport', u'techcrunch', u'techradar', u'the-economist', u'the-guardian-au', u'the-guardian-uk', u'the-hindu', u'the-huffington-post', u'the-lad-bible', u'the-new-york-times', u'the-next-web', u'the-sport-bible', u'the-telegraph', u'the-times-of-india', u'the-verge', u'the-wall-street-journal', u'the-washington-p

In [152]:
# Clasificar las fuentes por categorías

def mapping():
    d = {}
    response = requests.get('https://newsapi.org/v1/sources?language=en')
    response = response.json()
    for s in response['sources']:
        d[s['id']] = s['category']
    return d

In [153]:
#Revisando las categorias que tenemos
m = mapping()
print('category of reuters:', m['reuters'])
print('category of financial times:', m['financial-times'])

('category of reuters:', u'general')
('category of financial times:', u'business')


In [154]:
#Todas las categorias
print('categories:', list(set(m.values())))

('categories:', [u'gaming', u'business', u'entertainment', u'general', u'science-and-nature', u'music', u'politics', u'sport', u'technology'])


# GetDailyNews

La función principal es getDailyNews. Obtiene de la api las noticias y lo lleva a un dataframe y lo exporta a un csv file.
En cada iteración el csv file es actualizado y limpiado. Las redundancias son removidas por medio de la función cleanData
Para cada artículo recolectaremos las siguientes celdas:
- author
- title
- description
- url
- urlToImage
- publishedAt

Agregaremos otras:
- category
- scraping_date : La fecha en que se corrió el codigo



In [155]:
import requests
from bs4 import BeautifulSoup
from datetime import datetime
from functools import reduce

In [156]:
def getSources():
    source_url = 'https://newsapi.org/v1/sources?language=en'
    response = requests.get(source_url).json()
    sources = []
    for source in response['sources']:
        sources.append(source['id'])
    return sources

In [157]:
def mapping():
    d = {}
    response = requests.get('https://newsapi.org/v1/sources?language=en')
    response = response.json()
    for s in response['sources']:
        d[s['id']] = s['category']
    return d

In [158]:
def category(source, m):
    try:
        return m[source]
    except:
        return 'NC'


In [159]:
def cleanData(path):
    data = pd.read_csv(path)
    data = data.drop_duplicates('url')
    data.to_csv(path, index=False)

In [160]:
def connect():
    """ Connect to MySQL database """
    try:
        conn = mysql.connector.connect(host='localhost',
                                       database='NEWS',
                                       user='root',
                                       password='*******')
        if conn.is_connected():
            print('Connected to MySQL database')
 
    except Error as e:
        print(e)
 
    finally:
        conn.close()

In [161]:
def getDailyNews():
    sources = getSources()
    key = '741a381b7ce1463eb1fb6c41d42c68fd'
    url = ' https://newsapi.org/v1/articles?source={0}&sortBy={1}&apiKey={2}'
    responses = []
    for i, source in tqdm(enumerate(sources)):
        try:
            u = url.format(source, 'top',key)
            response = requests.get(u)
            r = response.json()
            for article in r['articles']:
                article['source'] = source
            responses.append(r)
        except:
            u = url.format(source, 'latest', key)
            response = requests.get(u)
            r = response.json()
            for article in r['articles']:
                article['source'] = source
            responses.append(r)
      
    news = pd.DataFrame(reduce(lambda x,y: x+y ,map(lambda r: r['articles'], responses)))
    news = news.dropna()
    news = news.drop_duplicates()
    d = mapping()
    news['category'] = news['source'].map(lambda s: category(s, d))
    news['scraping_date'] = datetime.now()

    try:
        aux = pd.read_csv('/Users/maritza/201 Data Analyst/news.csv')
    except:
        aux = pd.DataFrame(columns=list(news.columns))
        aux.to_csv('/Users/maritza/201 Data Analyst/news.csv', encoding='utf-8', index=False)

    with open('/Users/maritza/201 Data Analyst/news.csv', 'a') as f:
        news.to_csv(f, header=False, encoding='utf-8', index=False)

    cleanData('/Users/maritza/201 Data Analyst/news.csv')
    print('Done')

if __name__ == '__main__':
    getDailyNews()

60it [00:31,  1.91it/s]


Done


# Data analisys

In [162]:
pd.options.mode.chained_assignment = None
# nltk for nlp
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
# list of stopwords like articles, preposition
stop = set(stopwords.words('english'))
from string import punctuation
from collections import Counter
import re


In [163]:
data = pd.read_csv('./news.csv')

In [164]:
data

,author,description,publishedAt,source,title,url,urlToImage,category,scraping_date
0,http://www.abc.net.au/news/james-fettes/5652254,"With property prices at all-time highs, it is ...",2017-10-14T22:30:00Z,abc-news-au,Is buying an apartment a wise investment for y...,http://www.abc.net.au/news/2017-10-15/house-pr...,http://www.abc.net.au/news/image/9039908-1x1-7...,general,2017-10-14 19:48:19.393949
1,Kathryn Perrott,"Think everyone else is having sex, when you'd ...",2017-10-14T21:38:16Z,abc-news-au,Why low libido is the most common sex problem ...,http://www.abc.net.au/news/health/2017-10-15/w...,http://www.abc.net.au/cm/rimage/8794488-1x1-la...,general,2017-10-14 19:48:19.393949
2,http://www.abc.net.au/news/alisha-oflaherty/33...,A Perth mother claims she was unfairly targete...,2017-10-14T22:14:41Z,abc-news-au,Perth mum claims she was unfairly targeted by ...,http://www.abc.net.au/news/2017-10-15/mother-c...,http://www.abc.net.au/news/image/9046428-1x1-7...,general,2017-10-14 19:48:19.393949
3,http://www.abc.net.au/news/matthew-clayfield/4...,Sead Vrana started clearing land mines when he...,2017-10-14T19:08:24Z,abc-news-au,The country that may never be clear of land mines,http://www.abc.net.au/news/2017-10-15/bosnia-m...,http://www.abc.net.au/news/image/9046898-1x1-7...,general,2017-10-14 19:48:19.393949
4,http://www.abc.net.au/news/kathy-mcleish/4064014,Fifty years on from the first battle to save t...,2017-10-15T00:21:21Z,abc-news-au,Campaigners from 1960s return to fight for Gre...,http://www.abc.net.au/news/2017-10-15/great-ba...,http://www.abc.net.au/news/image/9050158-1x1-7...,general,2017-10-14 19:48:19.393949
5,http://www.abc.net.au/news/matthew-carney/6634066,President Xi is hoping to appoint four out of ...,2017-10-14T22:19:37Z,abc-news-au,President Xi's hyper-nationalism has people be...,http://www.abc.net.au/news/2017-10-15/presiden...,http://www.abc.net.au/news/image/8769310-1x1-7...,general,2017-10-14 19:48:19.393949
6,Al Jazeera,Many casualties feared as 'huge' car bomb expl...,2017-10-14T14:47:00Z,al-jazeera-english,Massive car bomb blast rocks Somalia's Mogadishu,http://www.aljazeera.com/news/2017/10/massive-...,http://www.aljazeera.com/mritems/Images/2017/1...,general,2017-10-14 19:48:19.393949
7,Al Jazeera,Syrian forces backed by the US are on the verg...,2017-10-14T20:51:00Z,al-jazeera-english,Evacuation deal as ISIL defeat in Syria's Raqq...,http://www.aljazeera.com/news/2017/10/evacuati...,http://www.aljazeera.com/mritems/Images/2017/1...,general,2017-10-14 19:48:19.393949
8,Al Jazeera,US President Donald Trump is refusing to endor...,2017-10-14T21:02:00Z,al-jazeera-english,What is next for Iran nuclear deal?,http://www.aljazeera.com/programmes/insidestor...,http://www.aljazeera.com/mritems/Images/2017/1...,general,2017-10-14 19:48:19.393949
9,David Child,The far-right FPO party expected to win up to ...,2017-10-12T11:02:00Z,al-jazeera-english,Austria's Election: What you need to know,http://www.aljazeera.com/news/2017/10/austria-...,http://www.aljazeera.com/mritems/Images/2017/1...,general,2017-10-14 19:48:19.393949


In [165]:
# Remover columnas duplicadas
data = data.drop_duplicates('description')
# Remover filas con valores nulos
data = data[~data['description'].isnull()]

In [166]:
data.shape

(997, 9)

In [167]:
#Filtramos aquellos en los que su descripcion es mayor a 140 caracteres,
#los demás son generalmente ruido
data['len'] = data['description'].map(len)
data = data[data.len > 140]
data.reset_index(inplace=True)
data.drop('index', inplace=True, axis=1)

In [168]:
data.shape

(338, 10)

In [169]:
data

,author,description,publishedAt,source,title,url,urlToImage,category,scraping_date,len
0,http://www.abc.net.au/news/alisha-oflaherty/33...,A Perth mother claims she was unfairly targete...,2017-10-14T22:14:41Z,abc-news-au,Perth mum claims she was unfairly targeted by ...,http://www.abc.net.au/news/2017-10-15/mother-c...,http://www.abc.net.au/news/image/9046428-1x1-7...,general,2017-10-14 19:48:19.393949,216
1,http://www.abc.net.au/news/kathy-mcleish/4064014,Fifty years on from the first battle to save t...,2017-10-15T00:21:21Z,abc-news-au,Campaigners from 1960s return to fight for Gre...,http://www.abc.net.au/news/2017-10-15/great-ba...,http://www.abc.net.au/news/image/9050158-1x1-7...,general,2017-10-14 19:48:19.393949,193
2,http://www.abc.net.au/news/matthew-carney/6634066,President Xi is hoping to appoint four out of ...,2017-10-14T22:19:37Z,abc-news-au,President Xi's hyper-nationalism has people be...,http://www.abc.net.au/news/2017-10-15/presiden...,http://www.abc.net.au/news/image/8769310-1x1-7...,general,2017-10-14 19:48:19.393949,200
3,JONATHAN J. COOPER,"SANTA ROSA, Calif. (AP) — Jack Weaver and his ...",2017-10-15T00:24:03Z,associated-press,Family dog emerges alive and happy from wildfi...,https://apnews.com/35749afa197f45bc930c3a398cb...,https://storage.googleapis.com/afs-prod/media/...,general,2017-10-14 19:48:19.393949,619
4,JONATHAN J. COOPER,"SANTA ROSA, Calif. (AP) — The warnings about i...",2017-10-14T19:41:44Z,associated-press,Coffey Park is Ground Zero for California fire...,https://apnews.com/5795bc803015486cbdbf96799d6...,https://storage.googleapis.com/afs-prod/media/...,general,2017-10-14 19:48:19.393949,661
5,CHRISTINA A. CASSIDY and MEGHAN HOYER,President Donald Trump's decision to end a pro...,2017-10-14T19:20:28Z,associated-press,Pro-Trump states most affected by his health c...,https://apnews.com/cd3fb39ebc2d47a8a3c636283ee...,https://storage.googleapis.com/afs-prod/media/...,general,2017-10-14 19:48:19.393949,579
6,ELLEN KNICKMEYER and TERRY CHEA,"SONOMA, Calif. (AP) — The California wildfires...",2017-10-14T23:10:13Z,associated-press,"Wildfires, now up to 100 miles wide, threaten ...",https://apnews.com/962f708143144d2eb356df65927...,https://storage.googleapis.com/afs-prod/media/...,general,2017-10-14 19:48:19.393949,607
7,ROB GILLIES and PATRICK LEJTENYI,"SMITHS FALLS, Ontario (AP) — A couple held hos...",2017-10-14T22:32:02Z,associated-press,Ex-hostages back in Canada after harrowing rai...,https://apnews.com/c650d65e8f6549d498967e483d3...,https://storage.googleapis.com/afs-prod/media/...,general,2017-10-14 19:48:19.393949,462
8,NASSER KARIMI and JON GAMBRELL,"TEHRAN, Iran (AP) — President Donald Trump's r...",2017-10-14T21:47:47Z,associated-press,Trump's speech sparks a new war of words betwe...,https://apnews.com/dcd29b043ce94cf5b436542acec...,https://storage.googleapis.com/afs-prod/media/...,general,2017-10-14 19:48:19.393949,707
9,NaN,LOS ANGELES (AP) — In a move virtually unprece...,2017-10-14T22:48:02Z,associated-press,Motion Picture Academy expels movie mogul Harv...,https://apnews.com/04b791b0e48e45c98592a9fd9f7...,https://storage.googleapis.com/afs-prod/media/...,general,2017-10-14 19:48:19.393949,555


# Text processing : tokenization

- Convertir enunciados en tokens
- Remover puntuación y conjunciones
- Quitar mayúsculas


In [189]:
def tokenizer(text):
    try:
        tokens_ = [word_tokenize(sent) for sent in sent_tokenize(text)]
        
        tokens = []
        for token_by_sent in tokens_:
            tokens += token_by_sent

        tokens = list(filter(lambda t: t.lower() not in stop, tokens))
        tokens = list(filter(lambda t: t not in punctuation, tokens))
        tokens = list(filter(lambda t: t not in ["'s", "n't", "...", "''", '``', 
                                            '2014', '2015', '2013'], tokens))
        filtered_tokens = []
        for token in tokens:
            if re.search('[a-zA-Z]', token):
                filtered_tokens.append(token)

        filtered_tokens = list(map(lambda token: token.lower(), filtered_tokens))

        return filtered_tokens
    except Error as e:
        print(e)

In [190]:
data.description

0      A Perth mother claims she was unfairly targete...
1      Fifty years on from the first battle to save t...
2      President Xi is hoping to appoint four out of ...
3      SANTA ROSA, Calif. (AP) — Jack Weaver and his ...
4      SANTA ROSA, Calif. (AP) — The warnings about i...
5      President Donald Trump's decision to end a pro...
6      SONOMA, Calif. (AP) — The California wildfires...
7      SMITHS FALLS, Ontario (AP) — A couple held hos...
8      TEHRAN, Iran (AP) — President Donald Trump's r...
9      LOS ANGELES (AP) — In a move virtually unprece...
10     DHAKA, Bangladesh (AP) — They had seen the ima...
11     The Milan derby was once a meeting of two of E...
12     Wilfried Zaha scores the winner as Crystal Pal...
13     Donald Trump boasted about triggering a declin...
14     U.S. Nafta negotiators are proposing to essent...
15     Puerto Rico’s efforts to record how many peopl...
16     The guardians of the world economy brushed asi...
17     Sonoma, Calif. (AP) -- T

In [191]:
tokenizer("Mary had a little lamb it's fleece was white as snow.")

['mary', 'little', 'lamb', 'fleece', 'white', 'snow']

In [243]:
desc['tokens']=tokenizer(str(data['description']))

In [226]:
tokenizer(str(data['description']))

['perth',
 'mother',
 'claims',
 'unfairly',
 'targete',
 'fifty',
 'years',
 'first',
 'battle',
 'save',
 'president',
 'xi',
 'hoping',
 'appoint',
 'four',
 'santa',
 'rosa',
 'calif.',
 'ap',
 'jack',
 'weaver',
 'santa',
 'rosa',
 'calif.',
 'ap',
 'warnings',
 'president',
 'donald',
 'trump',
 'decision',
 'end',
 'pro',
 'sonoma',
 'calif.',
 'ap',
 'california',
 'wildfires',
 'smiths',
 'falls',
 'ontario',
 'ap',
 'couple',
 'held',
 'hos',
 'tehran',
 'iran',
 'ap',
 'president',
 'donald',
 'trump',
 'r',
 'los',
 'angeles',
 'ap',
 'move',
 'virtually',
 'unprece',
 'dhaka',
 'bangladesh',
 'ap',
 'seen',
 'ima',
 'milan',
 'derby',
 'meeting',
 'two',
 'e',
 'wilfried',
 'zaha',
 'scores',
 'winner',
 'crystal',
 'pal',
 'donald',
 'trump',
 'boasted',
 'triggering',
 'declin',
 'u.s.',
 'nafta',
 'negotiators',
 'proposing',
 'essent',
 'puerto',
 'rico\xe2\x80\x99s',
 'efforts',
 'record',
 'many',
 'peopl',
 'guardians',
 'world',
 'economy',
 'brushed',
 'asi',
 'so

In [227]:
data.info

<bound method DataFrame.info of                                                 author  \
0    http://www.abc.net.au/news/alisha-oflaherty/33...   
1     http://www.abc.net.au/news/kathy-mcleish/4064014   
2    http://www.abc.net.au/news/matthew-carney/6634066   
3                                   JONATHAN J. COOPER   
4                                   JONATHAN J. COOPER   
5                CHRISTINA A. CASSIDY and MEGHAN HOYER   
6                      ELLEN KNICKMEYER and TERRY CHEA   
7                     ROB GILLIES and PATRICK LEJTENYI   
8                       NASSER KARIMI and JON GAMBRELL   
9                                                  NaN   
10                                         JULHAS ALAM   
11                                           BBC Sport   
12                                           BBC Sport   
13           Joe Sobczyk, Laura Litvan, Zachary Tracer   
14                          Josh Wingrove, Eric Martin   
15                                     N

In [232]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 338 entries, 0 to 337
Data columns (total 10 columns):
author           332 non-null object
description      338 non-null object
publishedAt      338 non-null object
source           338 non-null object
title            338 non-null object
url              338 non-null object
urlToImage       338 non-null object
category         338 non-null object
scraping_date    338 non-null object
len              338 non-null int64
dtypes: int64(1), object(9)
memory usage: 26.5+ KB


In [240]:
convertidor = lambda x: str(x[1:-1])
data['description'] = data.description.apply(convertidor)

In [241]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 338 entries, 0 to 337
Data columns (total 10 columns):
author           332 non-null object
description      338 non-null object
publishedAt      338 non-null object
source           338 non-null object
title            338 non-null object
url              338 non-null object
urlToImage       338 non-null object
category         338 non-null object
scraping_date    338 non-null object
len              338 non-null int64
dtypes: int64(1), object(9)
memory usage: 26.5+ KB


In [249]:
word_tokenize(str(data['description']))

['0',
 'Perth',
 'mother',
 'claims',
 'she',
 'was',
 'unfairly',
 'targeted',
 '...',
 '1',
 'fty',
 'years',
 'on',
 'from',
 'the',
 'first',
 'battle',
 'to',
 'save',
 'the',
 '...',
 '2',
 'esident',
 'Xi',
 'is',
 'hoping',
 'to',
 'appoint',
 'four',
 'out',
 'of',
 'th',
 '...',
 '3',
 'NTA',
 'ROSA',
 ',',
 'Calif.',
 '(',
 'AP',
 ')',
 '\xe2\x80\x94',
 'Jack',
 'Weaver',
 'and',
 'his',
 'br',
 '...',
 '4',
 'NTA',
 'ROSA',
 ',',
 'Calif.',
 '(',
 'AP',
 ')',
 '\xe2\x80\x94',
 'The',
 'warnings',
 'about',
 'imp',
 '...',
 '5',
 'esident',
 'Donald',
 'Trump',
 "'s",
 'decision',
 'to',
 'end',
 'a',
 'provi',
 '...',
 '6',
 'NOMA',
 ',',
 'Calif.',
 '(',
 'AP',
 ')',
 '\xe2\x80\x94',
 'The',
 'California',
 'wildfires',
 'r',
 '...',
 '7',
 'ITHS',
 'FALLS',
 ',',
 'Ontario',
 '(',
 'AP',
 ')',
 '\xe2\x80\x94',
 'A',
 'couple',
 'held',
 'hosta',
 '...',
 '8',
 'HRAN',
 ',',
 'Iran',
 '(',
 'AP',
 ')',
 '\xe2\x80\x94',
 'President',
 'Donald',
 'Trump',
 "'s",
 'ref',
 '..

In [247]:
data.set_index(['source'])

,author,description,publishedAt,title,url,urlToImage,category,scraping_date,len
source,,,,,,,,,
abc-news-au,http://www.abc.net.au/news/alisha-oflaherty/33...,Perth mother claims she was unfairly targeted ...,2017-10-14T22:14:41Z,Perth mum claims she was unfairly targeted by ...,http://www.abc.net.au/news/2017-10-15/mother-c...,http://www.abc.net.au/news/image/9046428-1x1-7...,general,2017-10-14 19:48:19.393949,216
abc-news-au,http://www.abc.net.au/news/kathy-mcleish/4064014,fty years on from the first battle to save the...,2017-10-15T00:21:21Z,Campaigners from 1960s return to fight for Gre...,http://www.abc.net.au/news/2017-10-15/great-ba...,http://www.abc.net.au/news/image/9050158-1x1-7...,general,2017-10-14 19:48:19.393949,193
abc-news-au,http://www.abc.net.au/news/matthew-carney/6634066,esident Xi is hoping to appoint four out of th...,2017-10-14T22:19:37Z,President Xi's hyper-nationalism has people be...,http://www.abc.net.au/news/2017-10-15/presiden...,http://www.abc.net.au/news/image/8769310-1x1-7...,general,2017-10-14 19:48:19.393949,200
associated-press,JONATHAN J. COOPER,"NTA ROSA, Calif. (AP) — Jack Weaver and his br...",2017-10-15T00:24:03Z,Family dog emerges alive and happy from wildfi...,https://apnews.com/35749afa197f45bc930c3a398cb...,https://storage.googleapis.com/afs-prod/media/...,general,2017-10-14 19:48:19.393949,619
associated-press,JONATHAN J. COOPER,"NTA ROSA, Calif. (AP) — The warnings about imp...",2017-10-14T19:41:44Z,Coffey Park is Ground Zero for California fire...,https://apnews.com/5795bc803015486cbdbf96799d6...,https://storage.googleapis.com/afs-prod/media/...,general,2017-10-14 19:48:19.393949,661
associated-press,CHRISTINA A. CASSIDY and MEGHAN HOYER,esident Donald Trump's decision to end a provi...,2017-10-14T19:20:28Z,Pro-Trump states most affected by his health c...,https://apnews.com/cd3fb39ebc2d47a8a3c636283ee...,https://storage.googleapis.com/afs-prod/media/...,general,2017-10-14 19:48:19.393949,579
associated-press,ELLEN KNICKMEYER and TERRY CHEA,"NOMA, Calif. (AP) — The California wildfires r...",2017-10-14T23:10:13Z,"Wildfires, now up to 100 miles wide, threaten ...",https://apnews.com/962f708143144d2eb356df65927...,https://storage.googleapis.com/afs-prod/media/...,general,2017-10-14 19:48:19.393949,607
associated-press,ROB GILLIES and PATRICK LEJTENYI,"ITHS FALLS, Ontario (AP) — A couple held hosta...",2017-10-14T22:32:02Z,Ex-hostages back in Canada after harrowing rai...,https://apnews.com/c650d65e8f6549d498967e483d3...,https://storage.googleapis.com/afs-prod/media/...,general,2017-10-14 19:48:19.393949,462
associated-press,NASSER KARIMI and JON GAMBRELL,"HRAN, Iran (AP) — President Donald Trump's ref...",2017-10-14T21:47:47Z,Trump's speech sparks a new war of words betwe...,https://apnews.com/dcd29b043ce94cf5b436542acec...,https://storage.googleapis.com/afs-prod/media/...,general,2017-10-14 19:48:19.393949,707


In [251]:
data['tokens'] = data['description'].map(tokenizer)

UnicodeDecodeError: 'ascii' codec can't decode byte 0xe2 in position 8: ordinal not in range(128)